# Image Colorization Final Project
Authors: Aret Tinoco, Keshav Gupta, Hal Halberstadt

Dataset: https://www.kaggle.com/datasets/darthgera/colorization

---

## Imports

We have to import some unsual libraries in order to get the RGB values of our target images into HSL format, and a few more for ease of viewing and on the same note ease displaying data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# import colorsys
from pathlib import Path
from PIL import Image # for resizing images

from skimage import io, color

import tensorflow as tf
from tensorflow.keras import models, layers, Input, Model, callbacks, utils, callbacks, optimizers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from IPython.display import display, HTML

Ease of reading

In [2]:
pd.set_option('display.max_columns', 200)
pd.options.display.width = 120
pd.options.display.max_colwidth = 50
display(HTML("<style>.container { width:100% !important; }</style>"))

Now to state the directory of the data to retrieve from.

In [3]:
data_dir = Path("C:/Users/smhal/Desktop/archive") 
img_shape = (512, 512, 3)
# shrink_shape = img_shape
shrink_shape = (256, 256, 3)
# shrink_shape = (128, 128, 3)
img_taken = 100

folder_paths = ['color', 'bw', 'color_val', 'bw_val']

---

## Useful Functions

We need a function to make conversion of each image easier

---

## Data Generator

Since we cannot hope to hold onto 18000 images in our kernel, we have to use a generator in order to be able to get data from the file and then train on that data.

In [4]:
class DataGenerator(utils.Sequence): 
    '''
    adapted from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
    
    Generates a color (rbg/hls) and black & white image for training data
    '''
    def __init__(self, ids, batch_size=1, 
                 dim=shrink_shape, n_channels=3, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = ids
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, *self.dim))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # B/W image
            img = load_img(ID)
            img = img.resize(shrink_shape[:2])
            img = np.array(img, dtype=np.float32)/255
            img = img ** 1/2.2 #rgb to srgb
            img = color.rgb2lab(img)
            
            X[i] = img
            
            # get location of color image
            color_ID = str(ID)
            color_ID = color_ID.replace("bw", "color")
            
            # target image
            img_y = load_img(color_ID)
            img_y = img_y.resize(shrink_shape[:2])
            img_y = np.array(img_y, dtype=np.float32)/255
            img_y = img_y ** 1/2.2 #rgb to srgb
            img_y = color.rgb2lab(img_y)
#             img_y = color.rgb2xyz(img_y)
#             img_y = color.xyz2lab(img_y)

            y[i] = img_y

        return X, y

In [5]:
# Dataset locations
bw_dir_train = data_dir / folder_paths[1]
# color_dir_train = data_dir / folder_paths[0]
#[:img_taken]
partition_bw = list(bw_dir_train.glob('*.jpg'))
# partition_color = list(color_dir_train.glob('*.jpg'))

# Generators
training_generator = DataGenerator(partition_bw)

In [6]:
# Dataset locations
bw_dir_val = data_dir / folder_paths[3]
color_dir_val = data_dir / folder_paths[2]

partition_bw = list(bw_dir_val.glob('*.jpg'))
partition_color = list(color_dir_val.glob('*.jpg'))

# Generators
validation_generator = DataGenerator(partition_bw)

---

## Model(s)

Next I want to read the data from the files and then just to make sure I am getting the right data from the right files

In [7]:
K.clear_session()  # Just for sanity
filter_size = 1
_padding = 'same'
_stride = 2
k_size = 3
_filters = 16
dropout_rate = 0.5
act_fun = layers.LeakyReLU(alpha=0.1)

In [8]:


inputs = Input(shrink_shape, name="img_input")
'''U-shape model'''

x = layers.Conv2D(3, k_size, activation=act_fun, padding=_padding)(inputs)
residual = x

# topmost start
x = layers.Conv2D(_filters, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)
x = layers.SpatialDropout2D(dropout_rate)(x)
x = layers.Conv2D(_filters, k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(_filters, k_size, activation=act_fun, padding=_padding)(x)
residual1 = x

# 2nd layer start
x = layers.Conv2D(_filters*2, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)
# x = layers.SpatialDropout2D(dropout_rate)(x)
x = layers.Conv2D(_filters*2, k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(_filters*2, k_size, activation=act_fun, padding=_padding)(x)
residual2 = x

# 3rd layer start
x = layers.Conv2D(_filters*3, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)
residual3 = x
x = layers.SpatialDropout2D(dropout_rate)(x)
x = layers.Conv2D(_filters*3, k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(_filters*3, k_size, activation=act_fun, padding=_padding)(x)

# Bottom of "U"/ Lowest layer
x = layers.Conv2D(_filters*6, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)
# x = layers.Conv2D(_filters*6, k_size, activation=act_fun, padding=_padding)(x)
x = layers.SpatialDropout2D(dropout_rate)(x)
x = layers.Conv2D(_filters*6, k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
# x = layers.Conv2D(_filters*6, k_size, activation=act_fun, padding=_padding)(x)
x = layers.Conv2DTranspose(_filters*6, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)

# # 3rd layer end
x = layers.Conv2D((_filters*3), k_size, activation=act_fun, padding=_padding)(x)
x = layers.Add()([residual3, x])
x = layers.SpatialDropout2D(dropout_rate)(x)
x = layers.Conv2D((_filters*4), k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose((_filters*3), k_size, strides=_stride, activation=act_fun, padding=_padding)(x)

# # 2nd layer end
x = layers.Conv2D(_filters*2, k_size, activation=act_fun, padding=_padding)(x)
x = layers.Add()([residual2, x])
x = layers.SpatialDropout2D(dropout_rate)(x)
x = layers.Conv2D(_filters*2, k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(_filters*2, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)

# # topmost end
x = layers.Conv2D(_filters, k_size, activation=act_fun, padding=_padding)(x)
x = layers.Add()([residual1, x])
x = layers.Conv2D(_filters, k_size, padding=_padding)(x)
x = layers.BatchNormalization(axis=3)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(_filters, k_size, strides=_stride, activation=act_fun, padding=_padding)(x)

# # outermost part
x = layers.Conv2D(3, k_size, activation=act_fun, padding=_padding)(x)
x = layers.Add()([residual, x])
x = layers.Conv2D(3, k_size, activation=act_fun, padding=_padding)(x)

model = Model(inputs, x)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_input (InputLayer)          [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 3)  84          img_input[0][0]                  
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 16) 448         conv2d[0][0]                     
__________________________________________________________________________________________________
spatial_dropout2d (SpatialDropo (None, 128, 128, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

---

## Training

Creating a loss function

In [10]:
my_monitor = "val_loss"
my_callbacks = [
    callbacks.EarlyStopping(monitor=my_monitor, patience=6, min_delta=0.2),
    callbacks.ReduceLROnPlateau(monitor=my_monitor, factor=0.3, patience=3, min_delta=0.2, verbose=1)
]

my_opt = optimizers.RMSprop(learning_rate=0.01, momentum=0.9)

In [ ]:
model.compile(optimizer=my_opt, loss='mean_absolute_error',  metrics=['accuracy'])

history = model.fit(training_generator, callbacks=my_callbacks, validation_data=validation_generator, epochs=100)

Epoch 1/100
 4222/18000 [======>.......................] - ETA: 7:31 - loss: 4.9822 - accuracy: 0.8458

---

## Example Results

Here is a few images that were colorized with this model

In [ ]:
n_img = 5
image_number = 3

In [ ]:
imgs_x = np.zeros((n_img, shrink_shape[0], shrink_shape[1], shrink_shape[2]), dtype='float32')
imgs_p = np.zeros((n_img, shrink_shape[0], shrink_shape[1], shrink_shape[2]), dtype='float32')
i = 0
for location in partition_bw[:n_img]:
    img = load_img(location)
    img = img.resize(shrink_shape[:2])
    img = np.array(img, dtype=np.float32)
    imgs_x[i] = img/255
    imgs_p[i] = img/255
    i += 1
    
plt.imshow(imgs_x[image_number]);

In [ ]:
output = model.predict(color.rgb2lab((imgs_p**1/2.2)))
# output = model.predict(color.rgb2lab((imgs_p)))
# output = model.predict(imgs_p)
# for imgs in output:
#     print(imgs[0][0])
# print(color.lab2rgb(imgs_p[image_number])[0][0])
output = color.lab2rgb(output[image_number]*2.2)

# output = color.lab2xyz(output[image_number])
# output = color.xyz2rgb(output)
# print(output[image_number][400:405])
# output = output

plt.imshow(output);

In [ ]:
imgs_y = np.zeros((n_img, shrink_shape[0], shrink_shape[1], shrink_shape[2]), dtype='float32')
i = 0
for location in partition_color[:n_img]:
    img = load_img(location)
    img = img.resize(shrink_shape[:2])
    img = np.array(img, dtype=np.float32)
    imgs_y[i] = img/255
    i += 1
    
plt.imshow(imgs_y[image_number]);

---

## Conclusions

Para1...